In [10]:
include("AuxiliaryFunctions/BornModel.jl")
include("AuxiliaryFunctions/Plotting.jl")
include("AuxiliaryFunctions/NoiseAuxiliaryFunctions.jl")

import .BornModel as BM
import .Plotting as P
import .NoiseAuxiliaryFunctions as NAF

using Plots, Plots.PlotMeasures
using DifferentialEquations
using BifurcationKit, Parameters
using Statistics
using KernelDensity
using JLD


In [11]:
# integration settings

spinup = 10
n_years = 10000 + spinup 

seed = 111

# lists with values for loop later on
σS = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
σF = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

6-element Vector{Float64}:
 0.0
 0.2
 0.4
 0.6
 0.8
 1.0

In [12]:
# heatmap settings and functions

clims = (0, 1)    # 0.4
cmap = cgrad(:viridis, rev=true); #:dense

function visualize_threshold!(R_matrix, threshold = 0.95)
    for i in 1:size(R_matrix, 1)
        for j in 1:size(R_matrix, 2)
            if R_matrix[i, j] < threshold
                x_coords = [σS[i] - 0.1/2, σS[i] - 0.1/2, σS[i] + 0.1/2, σS[i] + 0.1/2, σS[i] - 0.1/2, σS[i] + 0.1/2]
                y_coords = [σF[j] - 0.2/2, σF[j] + 0.2/2, σF[j] + 0.2/2, σF[j] - 0.2/2, σF[j] - 0.2/2, σF[j] + 0.2/2]

                plot!(x_coords, y_coords, color = :black, lw = 1.5, label = "")
            end
        end
    end
end

visualize_threshold! (generic function with 2 methods)

## F = 0.5 m/yr, S2 = 35 psu

In [4]:
# check if simulation exists; run if it doesn't, otherwise load data file
threshold = 21

if isfile("Data_21Sv/R_F05_S35_10000yr.jld") == false
    R_F05_S35 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(35), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(0.5), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F05_S35[i, j] = output.ratio
        end
    end 
    save("Data_21Sv/R_F05_S35_10000yr.jld", "R_F05_S35", R_F05_S35)
else 
    load("Data_21Sv/R_F05_S35_10000yr.jld")
end


In [5]:
p1 = heatmap(σS, σF, R_F05_S35', c = cmap, colorbar = false, widen = false, clims = clims, dpi = 300) 
visualize_threshold!(R_F05_S35)

title!("a) parameter set A")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F01_S35_10000yr.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F01_S35_10000yr.pdf"

## F = 1 m/yr, S2 = 35 psu

In [6]:
# check if simulation exists; run if it doesn't, otherwise load data_21Sv file
threshold = 21

if isfile("Data_21Sv/R_F1_S35_10000yr.jld") == false
    R_F1_S35 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(35), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(1.), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F1_S35[i, j] = output.ratio
        end
    end
    save("Data_21Sv/R_F1_S35_10000yr.jld", "R_F1_S35", R_F1_S35)
else 
    load("Data_21Sv/R_F1_S35_10000yr.jld")
end


In [7]:
p2 = heatmap(σS, σF, R_F1_S35', c = cmap, colorbar = false, widen = false, clims = clims, dpi = 300) 
visualize_threshold!(R_F1_S35)

title!("b) parameter set B")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F1_S35_10000yr.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F1_S35_10000yr.pdf"

## F = 1.5 m/yr, S2 = 35 psu

In [8]:
threshold = 21

if isfile("Data_21Sv/R_F15_S35_10000yr.jld") == false
    R_F15_S35 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(35), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(1.5), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F15_S35[i, j] = output.ratio
        end
    end
    save("Data_21Sv/R_F15_S35_10000yr.jld", "R_F15_S35", R_F15_S35)
else 
    load("Data_21Sv/R_F15_S35_10000yr.jld")
end


In [9]:
p3 = heatmap(σS, σF, R_F15_S35', c = cmap, colorbar = false, widen = false, clims = clims, dpi = 300) 
visualize_threshold!(R_F15_S35)

title!("c) parameter set C")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F15_S35_10000yr.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F15_S35_10000yr.pdf"

## F = 0.5 m/yr, S2 = 34.5 psu

In [13]:
threshold = 21

if isfile("Data_21Sv/R_F05_S3475_10000yr.jld") == false
    R_F05_S3475 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.75), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(0.5), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F05_S3475[i, j] = output.ratio
        end
    end
    save("Data_21Sv/R_F05_S3475_10000yr.jld", "R_F05_S3475", R_F05_S3475)
else 
    load("Data_21Sv/R_F05_S3475_10000yr.jld")
end

In [14]:
p4 = heatmap(σS, σF, R_F05_S3475', c = cmap, colorbar = false, widen = false, clims = clims, dpi = 300) 
visualize_threshold!(R_F05_S3475)

title!("d) parameter set D")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F05_S3475_10000yr.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F05_S3475_10000yr.pdf"

## F = 1 m/yr, S2 = 34.5 psu

In [15]:
threshold = 21

if isfile("Data_21Sv/R_F1_S3475_10000yr.jld") == false
    R_F1_S3475 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.75), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(1.), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F1_S3475[i, j] = output.ratio
        end
    end
    save("Data_21Sv/R_F1_S3475_10000yr.jld", "R_F1_S3475", R_F1_S3475)
else 
    load("Data_21Sv/R_F1_S3475_10000yr.jld")
end

In [16]:
p5 = heatmap(σS, σF, R_F1_S3475', c = cmap, colorbar = false, widen = false, clims = clims, dpi = 300) 
visualize_threshold!(R_F1_S3475)

title!("e) parameter set E")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F1_S3475_10000yr.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F1_S3475_10000yr.pdf"

## F = 1.5 m/yr, S2 = 34.5 psu

In [17]:
threshold = 21

if isfile("Data_21Sv/R_F15_S3475_10000yr.jld") == false
    R_F15_S3475 = zeros(length(σS), length(σF))

    for i in 1:length(σS)
        for j in 1:length(σF)
            noisevec = [0, BM.nondimensional_S(σS[i]), BM.nondimensional_F(σF[j])]

            pars = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.75), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0, 
                η = 1.29e2, μ1 = 0.838,μ2 = 83.8, μ3 = 12.1, μ4 = BM.nondimensional_F(1.5), r = 0.0714, τT0 = 0, τS2 = 1, τF = 90/365)
            output = NAF.integration(pars, noisevec, seed, n_years, spinup, threshold)
            
            R_F15_S3475[i, j] = output.ratio
        end
    end
    save("Data_21Sv/R_F15_S3475_10000yr.jld", "R_F15_S3475", R_F15_S3475)
else 
    load("Data_21Sv/R_F15_S3475_10000yr.jld")
end

In [18]:
p6 = heatmap(σS, σF, R_F15_S3475', c = cmap, colorbar = false, widen = false, tick_direction = :in, margin = 19px, clims = clims, dpi = 300) 
visualize_threshold!(R_F15_S3475)

title!("f) parameter set F")

xlabel!("σ\$_{S} \$ [psu]") # 
ylabel!("σ\$_{F} \$ [m yr\$^{-1}\$]")

xticks!(σS)
yticks!(σF)

savefig("Figures/heatmap_subplots/heatmap_F15_S3475_10000yr")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap_subplots\\heatmap_F15_S3475_10000yr.png"

## Combine in one plot

In [19]:
l = @layout[grid(2,3) a{0.05w}]

colorbar = heatmap((0:0.05:1).*ones(21,1), cmap = cmap, legend=:none, xticks=:none, yticks=(1:1:21, string.(0:0.05:1.00)), 
    ylabel = "R [-]", guide_position = :right, right_margin = 40px,
    tick_direction = :in)

p = plot(p1, p2, p3, p4, p5, p6, colorbar, 
    size = (width = 1200, height = 600), dpi = 300, layout = l,     
    # bottom_margin = 20px, left_margin = 30px, top_margin = 0px)
    bottom_margin = 25px, left_margin = 20px, top_margin = 10px)

savefig("Figures/heatmap.pdf")

"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\heatmap.pdf"